In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os,time
import DICOMPUTE as D

In [3]:
bucket = D.GC.get_bucket('ct-lymph-nodes')

Search completed after 1.546255350112915 seconds.


In [4]:
print(D.GC.get_bucket_size(bucket))

Getting size completed after 13.353251218795776 seconds.
57794974010


In [4]:
patients = sorted(D.GC.lsdir(bucket, 'dicom/'))

Listing directory completed after 3.893160104751587 seconds.


In [5]:
len(patients), patients[0]

(176, 'dicom/61.7.102432584850582937613178565576727689438/')

In [6]:
studies = sorted(D.GC.lsdir(bucket, patients[0]))

Listing directory completed after 0.9803445339202881 seconds.


In [7]:
len(studies), studies[0]

(1,
 'dicom/61.7.102432584850582937613178565576727689438/61.7.236153238415899774813399005901015124149/')

In [8]:
images = D.GC.download(bucket, studies[0], outdir='/mnt/data/ct-lymph-nodes/')

Download completed after 1.0281126499176025 seconds.


In [9]:
images[0]

'/mnt/data/ct-lymph-nodes/dicom/61.7.102432584850582937613178565576727689438/61.7.236153238415899774813399005901015124149/61.7.100084279359689996916137347486987360356.dcm'

In [23]:
#
# dcm2niix
#
dcm_dir = os.path.dirname(images[0])
nii_dir = dcm_dir.replace('dicom', 'nii')
nii_file = os.path.join(os.path.dirname(nii_dir), os.path.basename(dcm_dir))

nii_path = D.Processing.dcm2nii(dcm_dir, nii_file)

Conversion completed after  1.1783015727996826 seconds.


In [29]:
#
# 3D SIFT
#
feature_file = D.Processing.sift3d(nii_path)

KeyboardInterrupt: 